# This script shows how to use the various options for liquid-only thermometry
- This tutorial is also a great example for the other thermobarometers, as the flexbility of inputs is the same as for other phases

#### You need to install Thermobar once on your machine, if you haven't done this yet, uncomment the line below (remove the #)

In [1]:
#!pip install Thermobar

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Thermobar as pt

# Step 1: Load in data
- The function "import_excel" allows you to specify the name of the excel file and the sheet name This is where you load in data. E.g., specify excel file, and the sheet name
- Data should have the headings SiO2_Liq, TiO2_Liq etc. etc. The order of headings doesn't matter. 
- You can also have any other columns, e.g., estimate of pressure from any other proxy (melt inclusions, geophysics), and anything else you might want to plot (e.g., latitude, longitude)

In [3]:
out=pt.import_excel('Liquid_only_Thermometry.xlsx', sheet_name="Liquids")
my_input=out['my_input']
myLiquids1=out['Liqs']

## Inspect inputs to check you didn't have any funny column headings.
- It is always useful to inspect the outputs from import_excel, sometimes your column headings may have funny characters due to use of spaces, subscripts etc. in journal pdf tables. Check that all the columns you entered have numbers. If, say your SiO2_Liq heading had funny characters, this column will be filled with zeros when you inspect it.

In [4]:
myLiquids1.head()

,SiO2_Liq,TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,CaO_Liq,Na2O_Liq,K2O_Liq,Cr2O3_Liq,P2O5_Liq,H2O_Liq,Fe3Fet_Liq,NiO_Liq,CoO_Liq,CO2_Liq,Sample_ID_Liq
0,57.023602,0.623106,16.332899,4.36174,0.103851,4.19180,6.94858,3.59702,0.896895,0.000000,0.226584,5.59,0.0,0.0,0.0,0.0,0
1,57.658600,0.654150,17.194799,3.90621,0.084105,2.86892,5.91538,3.85948,1.018600,0.000000,0.214935,6.55,0.0,0.0,0.0,0.0,1
2,60.731201,0.862054,17.144199,4.07781,0.077488,2.50867,5.22075,4.45556,1.414160,0.000000,0.319638,3.14,0.0,0.0,0.0,0.0,2
3,61.532799,0.440860,16.508801,3.32990,0.037520,1.64150,4.34294,4.40860,1.407000,0.000000,0.215740,6.20,0.0,0.0,0.0,0.0,3
4,52.969101,0.803412,17.563000,5.93217,0.149472,3.78351,7.65110,3.80219,0.551178,0.037368,0.196182,6.58,0.0,0.0,0.0,0.0,4


## If at any point you need help with what functions inputs to use, you can type help(pt.function_name)

In [5]:
# Options for different thermometers laid out in the "equation option" = summary of advantages/disadvantages in Putirka 2008
help(pt.calculate_liq_only_temp)

Help on function calculate_liq_only_temp in module Thermobar.liquid_thermometers:

calculate_liq_only_temp(*, liq_comps, equationT, P=None, H2O_Liq=None)
     Liquid-only thermometery. Returns a temperature in Kelvin.
    
    Parameters
     -------
    
     liq_comps: DataFrame
         liquid compositions with column headings SiO2_Liq, MgO_Liq etc.
    
     equationT: str
         If has _sat at the end, represents the saturation surface of that mineral.
    
         Equations from Putirka et al. (2016).
             | T_Put2016_eq3_amp_sat (saturation surface of amphibole)
    
         Equations from Putirka (2008) and older studies:
    
             | T_Put2008_eq13
             | T_Put2008_eq14
             | T_Put2008_eq15
             | T_Put2008_eq16
             | T_Put2008_eq34_cpx_sat
             | T_Put2008_eq28b_opx_sat
             | T_Put1999_cpx_sat
             * Following 3 thermometers are adaptations of olivine-liquid thermometers with  DMg calculated using B

# Example 1 - P and H2O-independent thermometers
- The thermometer of Helz and Thornber 1987 is not dependent on pressure or H2O.
- Thus, the function only requires 2 inputs, the name of pandas dataframe with liquid compositions (here, myLiquids1) and the name of the equationT you have choosen
- All functions output in Kelvin and Kbar, so to get celcius, we subtract 273.15

In [6]:
T_Helz1987=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Helz1987_MgO")-273.15
# In jupyter lab/notebooks, outputs don't automatically display, but it prints the last line in each cell
T_Helz1987

0    1098.255182
1    1071.665294
2    1064.424269
3    1046.994150
4    1090.048550
5    1079.746499
6    1067.772325
7    1099.844089
8    1077.533086
9    1121.514497
Name: MgO_Liq, dtype: float64

# Example 2 - Pressure-dependent thermometer 
- a) If you select an equation which is Pressure-dependent, and don't specify a pressure, the code returns an error
- b) You can either select P="Solve" which returns a partial function. This means you can evaluate it at any pressure you want easily
- c) Or you can specify a fixed pressure, e.g., P=5 (saying run all calculations at 5 kbar)
- d) Or you can specify pressure based on a column in your original spreadsheet containing pressure

## Example 2a - Don't specify a pressure
- returns an error, telling you to input a pressure

In [7]:
Teq15_2kbar_6wt=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Put2008_eq15")-273.15 # Convert to Celcius
Teq15_2kbar_6wt

ValueError: T_Put2008_eq15 requires you to enter P, or specify P="Solve"

## Example 2b - specify P="Solve" to return a partial function

In [8]:
Teq15_partial=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Put2008_eq15", P="Solve")# Output is in Kelvin

In [9]:
# You can then evaluate this partial at any pressure you want (here, 10 kbar). This can save computation time
Teq15_10kbar=Teq15_partial(10)-273.15
Teq15_10kbar

0    1082.230315
1    1031.095340
2    1065.719538
3     991.644728
4    1049.453769
5    1033.581813
6    1021.013571
7     946.501846
8     919.539200
9    1108.248454
dtype: float64

## Example 2c - evaluate at constant pressure

In [10]:
# Here P=5 kbar
Teq15_5kbar=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Put2008_eq15", P=5)-273.15 # Output is in Kelvin
Teq15_5kbar

0    1062.650315
1    1011.515340
2    1046.139538
3     972.064728
4    1029.873769
5    1014.001813
6    1001.433571
7     926.921846
8     899.959200
9    1088.668454
dtype: float64

## Example 2d - Evaluate at values given by P_kbar column in inputted excel spreadsheet
- your column can be called anything, you just need state P=my_input['column name']. If you stored pressure in GPa say, you could do P=10*my_input['P_GPa']

In [11]:
Teq15_input=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Put2008_eq15", P=my_input['P_kbar'])-273.15 # Output is in Kelvin
Teq15_input

0    1050.902315
1    1003.683340
2    1046.139538
3     972.064728
4    1029.873769
5    1014.001813
6    1001.433571
7     926.921846
8     899.959200
9    1088.668454
dtype: float64

# Example 3 - Specifying water content
- the dataframe for liquids in tihs example has a H2O column. By default, H2O-dependent thermometers will use the values entered in this column
- However, you can also override this by specifying H2O_Liq=value in the function

In [12]:
Teq15_2H2O=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  
        equationT="T_Put2008_eq15", P=5, H2O_Liq=2)-273.15 # Output is in Kelvin
Teq15_2H2O

Water content replaced with that from H2O_Liq


0    1108.710017
1    1069.891842
2    1060.765739
3    1025.950726
4    1088.635168
5    1075.714113
6    1058.655371
7    1083.961043
8    1058.281402
9    1129.211252
dtype: float64

- We can also investigate how sensitive our calculations are to water, say we think maybe water ranges from 2-4 wt%, we can do the same calculation at 4 wt%

In [13]:
Teq15_4H2O=pt.calculate_liq_only_temp(liq_comps=myLiquids1,  equationT="T_Put2008_eq15", P=5, H2O_Liq=4)-273.15 # Output is in Kelvin
Teq15_4H2O

Water content replaced with that from H2O_Liq


0    1083.050017
1    1044.231842
2    1035.105739
3    1000.290726
4    1062.975168
5    1050.054113
6    1032.995371
7    1058.301043
8    1032.621402
9    1103.551252
dtype: float64

- We can now calculate the difference between these by subtracting one from the other. Here, as H2O_Liq is just a constant on the function, the difference is constant for all liquids

In [14]:
Teq15_4H2O-Teq15_2H2O

0   -25.66
1   -25.66
2   -25.66
3   -25.66
4   -25.66
5   -25.66
6   -25.66
7   -25.66
8   -25.66
9   -25.66
dtype: float64

# Exporting data
- There are a number of ways you can export data to excel
- The first example appends these new calculations onto a copy of the "my_input" dataframe. This means you can export a spreadsheet that looks exactly like your input spreadsheet, but with the values for the new calculations

In [17]:
# First make a copy of the dataframe so you don't overwrite the original
my_input_c=my_input.copy()
# To make a new column, you specify my_input_c['new column name']= value
my_input_c['T_Helz1987']=T_Helz1987
# You can add as many new columns as you wish
my_input_c['Teq15_4H2O']=Teq15_4H2O
# You can give your columns in the new dataframe any name you wish...
my_input_c['Temp eq 14 4 wt% H2O']=Teq15_4H2O
# Now, you ue the "to_excel" panda function to save this new appended dataframe to excel. The name of the dataframe you've been appended to goes on the let, and the name of the
# file you want to make goes bewteen the ' ' signs
my_input_c.to_excel('Thermometry_out1.xlsx')